In [4]:
%matplotlib inline
from matplotlib import pyplot as plt

# Getting Ready

In this chapter we'll use Guassian Processes for regression.  In the linear models section we saw how representing prior information on the coefficients was possible using Bayesian Ridge Regression.  Here we'll do something similar.  Instead of putting a prior on the coefficients we're going to put a prior on the functional form of the system and we're going to assume that prior is Gaussian.

Generally with a Guassian Process we assume the mean is 0, so it's the covariance function we'll need to specify.

So let's use some regression data and walk through how Gaussian Processes work on sklearn.

In [5]:
from sklearn.datasets import load_boston
boston = load_boston()

boston_X = boston.data
boston_y = boston.target

train_set = np.random.choice([True, False], len(boston_y), p=[.75, .25])

NameError: name 'np' is not defined

# How to do it

How that we have the data we'll create an sklearn GuassianProcess object.  By default it uses a constant regression function and squared exponential correlation.

In [ ]:
from sklearn.gaussian_process import GaussianProcess

gp = GaussianProcess()

gp.fit(boston_X[train_set], boston_y[train_set])

That's a formatible object definition.  A couple things to point out.

* `beta0` the regression weights, this defaults to such that MLE is used for estimation
* `corr`: this is the correlation function.  There are several built in correlation function.  We'll look at more of them in the "How it works" section
* `regr`: this is the constant regression function.
* `nugget`: This is the regularization parameter.  It defaults to a very small number.  You can either pass one value to be used for each data point or a single value to applied uniformly.

Ok so now that we've fit the object let's look at it's performance against the test object.

In [ ]:
test_preds = gp.predict(boston_X[~train_set])

Let's plot the predicted values verse the actual values, then because we're doing regression it's probably a good idea to look at a plotted residuals and histogram of the residuals.

In [ ]:
f, ax = plt.subplots(figsize=(10, 7), nrows=3)

f.tight_layout()

ax[0].plot(range(len(test_preds)), test_preds, label='Predicted Values');
ax[0].plot(range(len(test_preds)), boston_y[~train_set], label='Actual Values');
ax[0].set_title("Predicted vs Actuals")
ax[0].legend(loc='best')

ax[1].plot(range(len(test_preds)), test_preds - boston_y[~train_set]);
ax[1].set_title("Plotted Residuals")

ax[2].hist(test_preds - boston_y[~train_set]);
ax[2].set_title("Histogram of Residuals")

# How it works

Now that we've worked through a very quick example, let's look a little more about what some of the parameters are doing and how we can tune them based on the model we're tryin to fit.

First let's try to understand what's going on with the `corr` function.  This function describes the relationship between the different pairs of X.  Scikit-Learn offers 5 different correlation functions:

* absolute_exponential
* squared_exponential
* generalized_exponential
* cubic
* linear

For example, the squared expoential has the form:
$$
K = \exp(-{\frac{|d|^2}{2l^2}})
$$

Linear on the other hand is just dot product of the two points in question.

$$
K = x^T x^{'}
$$

Another parameter of interest is `theta0`.  This represents the starting point in the estimation of the of the parameters.

Once we have an estiamtion of K and the mean.  Our process if fully specied due to it being a Guassian Processes.  Therefore, we can now get the posterior predictive distribution which is used for prediction given unseen Xs.

Let's use a different `regr` function and apply a different `theta0` and look at how the predictions differ.

In [ ]:
gp = GaussianProcess(regr='linear', theta0=5e-1)

In [ ]:
gp.fit(boston_X[train_set], boston_y[train_set]);

In [ ]:
linear_preds = gp.predict(boston_X[~train_set])

In [ ]:
f, ax = plt.subplots(figsize=(7, 5))

f.tight_layout()

ax.hist(test_preds - boston_y[~train_set], label='Residuals Original', color='b', alpha=.5);
ax.hist(linear_preds - boston_y[~train_set], label='Residuals Linear', color='r', alpha=.5);
ax.set_title("Residuals")
ax.legend(loc='best')

Clearly the second model's predictions are better for the most part.  If we wanted to sum this up we could look at the MSE of the predictions.

In [ ]:
np.power(test_preds - boston_y[~train_set], 2).mean()

In [ ]:
np.power(linear_preds - boston_y[~train_set], 2).mean()

# There's More

We might want to understand the uncertainty in our estimates.  When we're making the predictions if we pass the argument `eval_MSE` as `True` we'll get the MSE as well as the predicted values.  This is done as a tuple during the return process.

In [ ]:
test_preds, MSE = gp.predict(boston_X[~train_set], eval_MSE=True)

In [ ]:
MSE[:5]

So now that we have errors in the estimates let's plot a the first few to get an indication of inaccuracy.

In [ ]:
f, ax = plt.subplots(figsize=(7, 5))

n = 20
rng = range(n)
ax.scatter(rng, test_preds[:n])
ax.errorbar(rng, test_preds[:n], yerr=1.96*MSE[:n])

ax.set_title("Predictions with Error Bars")

ax.set_xlim((-1, 21));

As you can see there's quite a bit of variance in the estimates for a lot of these points.  On the other hand our overall error wasn't too bad.